# Website Backend Notebook

This notebook is launched from the ADHD Tools website. It mounts your Google Drive and can read or write files based on parameters passed in the URL.


In [ ]:
from google.colab import drive, output
import os, json

# Parse query parameters
params = json.loads(output.eval_js(
    "JSON.stringify(Object.fromEntries(new URLSearchParams(window.location.search)))"
))
print('Params:', params)


In [ ]:
drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/ADHDtools'
os.makedirs(DATA_DIR, exist_ok=True)


In [ ]:
def load_data(name, default=None):
    path = os.path.join(DATA_DIR, name)
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    return default if default is not None else {}

def save_data(name, data):
    with open(os.path.join(DATA_DIR, name), 'w') as f:
        json.dump(data, f)

def merge_by_timestamp(existing, incoming):
    log_key = 'adhd-storage-log'
    existing_log = existing.get(log_key, {})
    incoming_log = incoming.get(log_key, {})
    merged = {**existing, **incoming}
    merged_log = {**existing_log, **incoming_log}
    for key, ts in incoming_log.items():
        old_ts = existing_log.get(key)
        if not old_ts or ts > old_ts:
            merged[key] = incoming.get(key)
    merged[log_key] = merged_log
    return merged


In [ ]:
filename = params.get('filename', 'webdata.json')
action = params.get('action', 'read')

if action == 'write':
    raw = params.get('data', {})
    if isinstance(raw, str):
        try:
            raw = json.loads(raw)
        except Exception:
            try:
                import base64, urllib.parse
                decoded = base64.urlsafe_b64decode(raw).decode()
                raw = json.loads(decoded)
            except Exception:
                raw = {}
    data = raw
    existing = load_data(filename, {})
    merged = merge_by_timestamp(existing, data)
    save_data(filename, merged)
    print(f'Saved to {filename}')
else:
    data = load_data(filename, {})
    print(f'Loaded from {filename}:', data)
